<a href="https://colab.research.google.com/github/mrchapagain/ConsumerDataAnalytics/blob/main/Data_Exploration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Here i try to explore different dataset to look if they can be connected, linked or somehow can talk to eachother.**
*   *Data by Frederik*
*   *Frida dataset of Nutrient content of food items*
*   *Reciept from my personal Storebox account*
*   *link from TradeSync data from GS1*
*   *Climate Data from Concito*

In [18]:
!pip install PyPDF2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [22]:
# Necessary libaries
import pandas as pd
import os
#import PyPDF2

# Data by Frederik

***Columns description***

*   **itemname:** name provided by the merchant. It often, but not always, contains a enough information to ID the product.  Sometimes weight, or other information is also provided here.
*   **itemnumber:** the product itemnumber. It can be a standardized GTIN 13 number ( 12-13 digits) supported by GS1 or a non standard number created by the retailer to identify i.e. Bananas
*   **category:** a category code not sure how to interpret and often missing
*   **percent_purchases:** Percent off all receipt lines in the dataset where each line is a purchas

In [23]:
# Import data
path_iss_data="https://github.com/mrchapagain/ConsumerDataAnalytics/raw/main/cpd_summary.csv"

# Open data as csv file
data_iss= pd.read_csv(path_iss_data, sep= ";", index_col=False, encoding='latin-1')

# lets see frist 3 and last 3 rows od the dataset
data_iss.head(3).append(data_iss.tail(5))

,itemname,itemnumber,category,percent_purchases
0,PANT,8880171,845,"3,369"
1,PANT,8880171,NaN,"2,055"
2,PANT,8880172,845,"1,497"
112761,PINOTEX SUPERDEC SOR,61009053404,6100,"-0,002"
112762,QUICK GEVINSTER,214196,926,"-0,002"
112763,UKRUDTSDUG BASIC 1X,42139019916,4213,"-0,002"
112764,TRAPPEVANGE,5708832546219,600,"-0,004"
112765,BR MÆRKER RETUR,5766183013538,21,"-0,005"


In [24]:
# Only print one selective row of interest to deep deeper
#data_iss[data_iss.itemname == "BANANER, LØSE" or "ØKO BANANER OR LØSE" or "BANANER"] # or "ØKO BANANER, LØSE", HAVFRISK LAK, APPELSINER 2 KG,Agurk

# only some of the rows of different kinds of the banana
data_iss.iloc[[11, 12, 14, 18], :]   

,itemname,itemnumber,category,percent_purchases
11,"ØKO BANANER, LØSE",5712873336944,89,"0,567"
12,"BANANER,LØSE",5711044628659,NaN,"0,565"
14,BANANER,406,0000000030,"0,419"
18,"BANANER,LØSE",5711044628659,89,"0,266"


***In above ddataset, only one food itemname (fx `"ØKO BANANER, LØSE"`) is appear in different rows with;***
*   Many different itemnumber, even some are 13 digits and other are 8 digits
*   NULL percent purchage

***Here i like to use NLP to find split itemname to name and other entity***

In [40]:
#!pip install nltk

In [57]:
# Liaries for NLP
from nltk.tokenize import sent_tokenize, word_tokenize

import nltk
nltk.download("punkt")
nltk.download("stopwords")
from nltk.corpus import stopwords

from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
nltk.download("omw-1.4")

stop_words= set(stopwords.words("danish"))
print(stop_words)

{'men', 'noget', 'efter', 'disse', 'ind', 'anden', 'blev', 'ikke', 'jeg', 'den', 'mod', 'dem', 'selv', 'der', 'de', 'alt', 'sin', 'en', 'bliver', 'sine', 'hans', 'deres', 'og', 'som', 'mine', 'er', 'havde', 'op', 'skal', 'ned', 'eller', 'skulle', 'os', 'under', 'denne', 'at', 'det', 'været', 'ville', 'blive', 'hendes', 'vil', 'nogle', 'mit', 'på', 'meget', 'nu', 'sit', 'hvis', 'han', 'hende', 'med', 'når', 'hvor', 'sådan', 'har', 'dig', 'thi', 'vor', 'om', 'dog', 'ud', 'din', 'i', 'du', 'hvad', 'også', 'her', 'mange', 'hun', 'sig', 'fra', 'dette', 'af', 'over', 'var', 'man', 'end', 'ham', 'da', 'vi', 'ad', 'have', 'min', 'mig', 'jo', 'for', 'et', 'være', 'alle', 'jer', 'kunne', 'til', 'hos'}


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [53]:
# Word Tokenization
tokenized_item = word_tokenize("	ØKO BANANER, LØSE ")
tokenized_item


['ØKO', 'BANANER', ',', 'LØSE']

In [58]:
#Lexicon Normalization

lem = WordNetLemmatizer()
lem_items= []


for sent in tokenized_item:
  lem_items.append(lem.lemmatize(sent))

# for w in filtered_sentence:
#     lem_words.append(lem.lemmatize(filtered_sentence,"v"))
                                 
print("Lemmatized Word:", lem_items)
#print("Stemmed Word:", stemmed_words)

LookupError: ignored

# FRIDA Food Composition Database
***FRIDA Food Composition Database  contain mainly Nutrition value of the each food items***

*   ***Category list:*** https://frida.fooddata.dk/food/lists/grouped?lang=en
*   ***Data display inspiration:*** https://foodb.ca/foods/FOOD00011
*   ***Display on FRIDA:*** https://frida.fooddata.dk/food/754?lang=en

In [26]:
# Import FRIDA Nutrient data
path_frida_data="https://github.com/mrchapagain/ConsumerDataAnalytics/raw/main/FridaNutrientData.xlsx"

# Lets open data as they are but only 26 columns out of 201 columns
data_frida= pd.read_excel(path_frida_data, sheet_name=1, index_col=0, header=0, skiprows=0).round(decimals = 2).iloc[:, :25]
data_frida.head(3)

,Unnamed: 1,Unnamed: 2,Svind,"Energi, kJ","Energi, kcal",Nitrogen-til-protein faktor,"Nitrogen, total","Protein, videnskabelig","Protein, deklaration","Kulhydrat, difference",...,FCF,Alkohol,Aske,Tørstof,Vand,A-vitamin,Retinol,beta-caroten,D-vitamin,D3-vitamin
Nummer,gruppe,Navn,%,kJ,kcal,-,g,g,g,g,...,-,g,g,g,g,RE,µg,µg,µg,µg
1,Bærfrugt,"Jordbær, rå",4,162,38,6.25,0.106,0.7,0.7,8.3,...,0.8,0,0.4,10,90,3.33,0,40,0,0
2,Kernefrugt,"Æble, uspec., råt",10,233,55,6.25,0.043,0.3,0.3,14.3,...,0.8,0,0.3,15.1,84.9,2.08,0,25,0,0


In [33]:
# Lets define columns name that are readeable
column_names= ['Gruppe', 'Navn', 'Svind_%', 'Energy_kj', 'Energy_kcal', 'Nitrogen_til_protein_faktor', 'Nitrogen_total_g', 'Protein_videnskabelign_g', 'Protein_deklaration_g', 'Kulhydrat_difference_g', 'Kulhydrat_tilgængelig_g', 'Kulhydrat_deklaration_g', 'Tilsat_sukker_g', 'Kostfibre_g', 'Fedt_total_g', 'FCF', 'Alkohol_g', 'Aske_g', 'Tørstof_g', 'Vand_g', 'A_vitamin_RE', 'Retinol_µg', 'beta_caroten_µg', 'D_vitamin_µg', 'D3_vitamin_µg']

# Lets open with little modificaton so easy to read and understand
data_frida= pd.read_excel(path_frida_data, sheet_name=1, index_col=0, header=None, skiprows=2).round(decimals = 2).iloc[:, :25].reset_index(drop=True)
# Assign columns name that defined just above
data_frida.columns= column_names

# lets see frist 3 and last 3 rows od the dataset
data_frida.head(3).append(data_frida.tail(3))
data_frida[data_frida.Gruppe == 'Sydfrugt'] #"Frugt"-grøntsager, Sydfrugt, Bærfrugt
#data_frida[data_frida.Navn == "Banan, rå"]


,Gruppe,Navn,Svind_%,Energy_kj,Energy_kcal,Nitrogen_til_protein_faktor,Nitrogen_total_g,Protein_videnskabelign_g,Protein_deklaration_g,Kulhydrat_difference_g,...,FCF,Alkohol_g,Aske_g,Tørstof_g,Vand_g,A_vitamin_RE,Retinol_µg,beta_caroten_µg,D_vitamin_µg,D3_vitamin_µg
2,Sydfrugt,"Banan, rå",41,396,93,6.25,0.183,1.1,1.1,22.6,...,0.73,0,0.7,24.6,75.3,4.42,0,53,0,0
19,Sydfrugt,"Citron, rå",35,187,44,6.25,0.08,0.5,0.5,9.9,...,0.8,0,0.3,11.8,88.2,1.67,0,20,0,0
34,Sydfrugt,"Vandmelon, rå",34,140,33,6.25,0.128,0.8,0.8,7.5,...,0.7,0,0.3,8.7,91.3,1.67,0,20,0,0
66,Sydfrugt,"Appelsin, rå",28,205,49,6.25,0.142,0.9,0.9,12.2,...,0.8,0,0.4,13.6,86.4,4,0,48,0,0
166,Sydfrugt,"Tamarind, rå",66,1086,256,6.25,0.448,2.8,2.8,62.5,...,0.7,0,2.7,68.6,31.4,1.5,0,18,0,0
196,Sydfrugt,"Citronsaft, friskpresset",0,134,31,6.25,0.061,0.4,0.4,8.6,...,0.8,0,0.3,9.3,90.7,0.25,0,3,0,0
243,Sydfrugt,"Figen, tørret",0,1234,292,6.25,0.528,3.3,3.3,72.2,...,0.8,0,1.9,78.3,21.7,6.67,0,80,0,0
268,Sydfrugt,"Loquat, japansk mispel, rå",38,206,49,6.25,0.069,0.4,0.4,12.1,...,0.7,0,0.5,13.3,86.7,iv,iv,iv,0,iv
274,Sydfrugt,"Honningmelon, rå",39,113,27,6.25,0.088,0.6,0.6,6,...,0.7,0,0.5,7.3,92.7,1.38,0,16.5,0,0
308,Sydfrugt,"Netmelon, rå",30,99,23,6.25,0.131,0.8,0.8,5.2,...,0.7,0,0.5,6.7,93.3,91.7,0,1100,0,0


In [28]:
row=data_frida[data_frida.Navn== "Banan, rå"]   # Kylling, bryst (filet), kogt, pålæg,  ØKO BANANER, LØSE
print("Id: ", row.index[0])
print("Category: ", row.Gruppe.values[0])
print("Name: ", row.Navn.values[0])
print("Total Energy: ", row.Energy_kj.values[0], "Kj (", row.Energy_kcal.values[0], "Kacl)")
print("Total Energy: ", row.Energy_kcal.values[0])
print("Total Energy: ", row.Energy_kj.values[0])

Id:  2
Category:  Sydfrugt
Name:  Banan, rå
Total Energy:  396 Kj ( 93 Kacl)
Total Energy:  93
Total Energy:  396


In [ ]:
# To see NAN value and data types of each columns
#data_frida.info()

***Above dataset do not contain NAN value and all of the columns are object type, so some of the columns need to be changed to interger or float data type to work as numbers value***

# Recipt from my personal Storebox account
- This data set i download from my own storebox account. Lets see what kind of information we can extract from storebox account

***The Store-box dataset contain one dataset of recieptand another dataset of the actual list of itemspurchaged each time***

In [ ]:
#Import data
path_storebox_data= "https://github.com/mrchapagain/ConsumerDataAnalytics/raw/main/storbox_recipts.json"

#Loading dat in dataframe 
data_storebox= pd.read_json(path_storebox_data)
# ['id', 'type', 'receiptId', 'purchaseDate', 'purchaseDateTimeString', 'orderNumber', 'price', 'merchant', 'receiptLines', 'terminalId', 'payments', 'cards', 
#     'barcode', 'headerTexts', 'footerTexts', 'userIds', 'imageId', 'receiptImage', 'note', 'tags', 'headerText', 'footerText', 'purchaseDateTimeZoneOffset', 'legalReceipt']

# Lets open only columns that make sence
data_storebox= data_storebox[["id", "receiptId", "purchaseDate", "orderNumber", "price", "merchant",'barcode']]
data_storebox.head(3)

***With the receiptId i can go to indiviisual reciept and extract list of purchage items. However the reciept is in PDF so need little advance RegEx code so i have not done it yet but will update soon.***


**Now lets  open reciept of  of the very first column (receiptId= "05jrqpia3ofdzsbq773yme4on0ko1f87") of above data**

In [ ]:
# Lets open reciept of very first columns (receiptId= "05jrqpia3ofdzsbq773yme4on0ko1f87") of above dataset (data_storbox)
data_storbox_reciept= "/content/05jrqpia3ofdzsbq773yme4on0ko1f87-14736470631022876123.pdf" # "https://github.com/mrchapagain/food_ontology/raw/main/storbox_recipt_pdf.pdf"

# read information from PDF file (open it in read binary mode)
pdfFileObj = open(data_storbox_reciept, 'rb')

# call and store PdfFileReader
# object in pdfReader
pdfReader = PyPDF2.PdfFileReader(pdfFileObj)

# to access first page if the PDF file have multiple pages, pass 0
pageObj = pdfReader.getPage(0)

# extract the page object by extractText() function
texts = pageObj.extractText()

# print the extracted texts
print(texts)

***I have extracted data from PDF file now first need to turn it to dictionary and then DataFrame.***

In [ ]:
lines= texts.rstrip("\n")
lines

# Climate Data from Concito

In [ ]:
#Import data
path_climate_data="https://github.com/mrchapagain/ConsumerDataAnalytics/raw/main/ClimateData.xlsx"

# Loading the data into the data-frame
col_names= ["id", "Product_dk", "Category_dk", "Product_en", "Category_en", "Unit", "Agriculture", "iLUC", "Processing", "Packaging", "Transport", "Retail", "Total_CO2_eq/kg", "Energy_KJ", "Fat_g", "Carb_g", "Protein_g", "Data_Source", "Comments", "GPC_Level4_en", "ID_Food", "ID_Pack", "ID_Retail", "GPC_Category_en", "GPC_Category_dk", "GPC_Level4_dk", "product_type", "GPC_level1", "Food_group", "GPC_level2", "Un/Processed", "GPC_Level3", "Extra_category"]

# Original columns name are replaced with ralavant names (col_names)
data_climate= pd.read_excel(path_climate_data, sheet_name=1, index_col=False, header=None, skiprows=1, names= col_names).round(decimals = 2)
data_climate.head(3)

***The reason to open this climate dataset is to see if it is possible link other dataset with column name "GPC_Level" which is releted to GS1 Trade Sync code.***

**Let´s have a look dataset with only selected columns that might make sense\**

In [ ]:
cols_to_use= ["Product_en", "Category_en", "ID_Food", "ID_Pack", "ID_Retail", "GPC_level1", "GPC_level2", "GPC_Level3", "GPC_Level4_dk", "Fat_g", "Carb_g", "Protein_g", "Energy_KJ", "Agriculture", "iLUC", "Processing", "Packaging", "Transport", "Retail", "Total_CO2_eq/kg"]

data_climate_selected= data_climate.loc[:, cols_to_use].round(decimals = 2)
data_climate_selected.head()

# Data from COOP store (store data)

In [ ]:
path_coop_data= "https://raw.githubusercontent.com/mrchapagain/ConsumerDataAnalytics/main/storedata/storedata_csv.csv"
#data_coop= pd.read_excel(path_coop_data, sheet_name=1, header=0, skiprows=0).round(decimals = 2)

data_coop= pd.read_csv(path_coop_data, index_col=0, sep= ";", on_bad_lines='skip', engine='python')

data_coop.head()

***These information are saved wrong way; all the columns of csv in one column so it open like one column***

So i am going to open with string split method with comma " ,"

# TradeSync data from GS1 

In [ ]:
import os
import xml.etree.ElementTree as ET
from bs4 import BeautifulSoup

In [ ]:
#Import data
path_tradesync_data= "/content/testdata_gs1.xml" # "https://github.com/mrchapagain/food_ontology/blob/main/gs1_data/testdata.xml"

In [ ]:
tree = ET.parse(path_tradesync_data)
root = tree.getroot()
print("Roots:", root.tag)
print("Attrib:", root.attrib)
print("Childerns: ", list(root))


***This part is hidden because the datast is very big and  take somany lines***

In [ ]:
# Code to have a look whole data
"""# Reading the data inside the xml file to a variable under the name  data
with open(path_tradesync_data, 'r') as f:
    for line in f.readlines():
      print(line)
"""

# Code to explore tags
"""# The xml file has a LOT of tags!
list_of_items= []
stopAt = 200
for elem in root.iter():
    print(elem.tag)
    list_of_items.append(elem.tag)
    #print(elem.attrib)
    if stopAt==0:
        break
    stopAt -= 1"""

In [ ]:
#print(ET.tostring(root, encoding='utf8').decode('utf8'))
list_of_items_to_find= ["entityIdentification", "brandName", "functionalName", "gln", "gtin", "creationDateTime", "dataRecipient", "contactDescription", "partyAddress", "gpcCategoryCode", "grossWeight", "descriptionShort", "allergenSpecificationName", "ingredientStatement", "tradeItemTemperatureConditionTypeCode", "uniformResourceIdentifier"]
#  "additionalTradeItemIdentification", "partyName",

#Lets start the dictionary to collecct items
dict_item_text = {}

# Loop over the tags that make sense (from list above)
for list_item in list_of_items_to_find:
  
  # Again  looping to get the text value
  for item in root.iter(list_item):
    dict_item_text[list_item] = item.text

# Lets see what we have c collected from the loop
dict_item_text

***Above dictinary are the information extracted from GS1 xml file associated with each item from the purchage list***

*We can grasp a lot of infoation about the item*